<https://github.com/PolymathicAI/xVal>


In [ ]:
from flax import nnx
import flax

flax.__version__

In [2]:
import ast
import re
from datetime import datetime as dt
import os

import icecream
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import optax
from flax.struct import dataclass
from icecream import ic
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from transformers import BertTokenizerFast, FlaxBertModel

import hephaestus as hp

icecream.install()
ic_disable = False  # Global variable to disable ic
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)
pd.options.mode.copy_on_write = True

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = FlaxBertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Get the embeddings matrix
embeddings = model.params["embeddings"]["word_embeddings"]["embedding"]

# Now you can access specific embeddings like this:
# For example, to get embeddings for tokens 23, 293, and 993:
selected_embeddings = jnp.take(embeddings, jnp.array([23, 293, 993]), axis=0)

# If you want to get embeddings for specific words:
words = ["hello", "world", "example"]
tokens = tokenizer.convert_tokens_to_ids(words)
word_embeddings = jnp.take(embeddings, jnp.array(tokens), axis=0)
word_embeddings.shape

In [5]:
def line2df(line, idx):
    data_rows = []
    line = ast.literal_eval(line)
    for i, time_step in enumerate(line["data"]):
        row = {"time_step": i}
        # Add position data for each planet
        for j, position in enumerate(time_step):
            row[f"planet{j}_x"] = position[0]
            row[f"planet{j}_y"] = position[1]
        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    description = line.pop("description")
    step_size = description.pop("stepsize")
    for k, v in description.items():
        for k_prop, v_prop in v.items():
            df[f"{k}_{k_prop}"] = v_prop
    df["time_step"] = df["time_step"] * step_size
    df.insert(0, "idx", idx)

    return df

In [ ]:
files = os.listdir("data")
if "planets.parquet" not in files:
    with open("data/planets.data") as f:
        data = f.read().splitlines()

        dfs = []
        for idx, line in enumerate(tqdm(data)):
            dfs.append(line2df(line, idx))
        df = pd.concat(dfs)
    df.to_parquet("data/planets.parquet")
else:
    df = pd.read_parquet("data/planets.parquet")


# Combine total mass of all planets into one column `planet<n>_m`
mass_regex = re.compile(r"planet(\d+)_m")
mass_cols = [col for col in df.columns if mass_regex.match(col)]
df["total_mass"] = df[mass_cols].sum(axis=1)
# df = df.reset_index(drop=True)
# Introduce categorical columns for the number of planets choose non null columns with mass
df["n_planets"] = df[mass_cols].notnull().sum(axis=1).astype("object")
df["n_planets"] = df["n_planets"].apply(lambda x: f"{x}_planets")
# Create category acceleration if the sum of plane/d_[x,y, z] is greater than 0
df["acceleration_x"] = df[
    [col for col in df.columns if "planet" in col and "_x" in col]
].sum(axis=1)
# Set acceleration_x to "increasing" if greater than 0 else "decreasing"
df["acceleration_x"] = (
    df["acceleration_x"]
    .apply(lambda x: "increasing" if x > 0 else "decreasing")
    .astype("object")
)
df["acceleration_y"] = df[
    [col for col in df.columns if "planet" in col and "_y" in col]
].sum(axis=1)
df["acceleration_y"] = df["acceleration_y"].apply(
    lambda x: "increasing" if x > 0 else "decreasing"
)


df.describe()

In [ ]:
df.head()

In [ ]:
df_categorical = df.select_dtypes(include=["object"]).astype(str)
unique_values_per_column = df_categorical.apply(
    pd.Series.unique
).values  # .flatten().tolist()
flattened_unique_values = np.concatenate(unique_values_per_column).tolist()
unique_values = list(set(flattened_unique_values))
unique_values

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer.vocab_size

In [ ]:
df.select_dtypes(include="object").groupby(
    df.select_dtypes(include="object").columns.tolist()
).size().reset_index(name="count")

In [12]:
df = df.reset_index(drop=True)

In [ ]:
# Get train test split at 80/20
time_series_config = hp.TimeSeriesConfig.generate(df=df)
train_idx = int(df.idx.max() * 0.8)
train_df = df.loc[df.idx < train_idx].copy()
test_df = df.loc[df.idx >= train_idx].copy()
# del df
train_ds = hp.TimeSeriesDS(train_df, time_series_config)
test_ds = hp.TimeSeriesDS(test_df, time_series_config)
len(train_ds), len(test_ds)

In [ ]:
len(time_series_config.numeric_col_tokens) + len(
    time_series_config.categorical_col_tokens
)

In [38]:
def make_batch(ds: hp.TimeSeriesDS, start: int, length: int):
    numeric = []
    categorical = []
    for i in range(start, length + start):
        numeric.append(ds[i][0])
        categorical.append(ds[i][1])
    # print index of None values
    return {"numeric": jnp.array(numeric), "categorical": jnp.array(categorical)}


batch = make_batch(train_ds, 0, 4)
# print(batch1["numeric"].shape, batch1["categorical"].shape)
# (4, 27, 59) (4, 3, 59)
# batch

In [16]:
multiplier = 4
time_series_regressor = hp.TimeSeriesDecoder(
    time_series_config, d_model=512, n_heads=8 * multiplier, rngs=nnx.Rngs(0)
)
# nnx.display(time_series_regressor)

In [ ]:
embed = nnx.Embed(
    num_embeddings=time_series_config.n_tokens, features=512, rngs=nnx.Rngs(0)
)
# indices_input = jnp.array([[0, 1, 2], [-1, -2, -3]])
indices_input = jnp.array([0, 1])
embed(indices_input)

In [ ]:
token_reservoir_lookup = time_series_config.reservoir_encoded[1]
embed(token_reservoir_lookup)

In [ ]:
token_reservoir_lookup

In [ ]:
time_series_regressor.time_series_transformer.embedding(jnp.array([-1, -2, -3]))

In [39]:
res = time_series_regressor(
    numeric_inputs=batch["numeric"],
    categorical_inputs=batch["categorical"],
    deterministic=False,
)

In [40]:
res["numeric_out"].shape, res["categorical_out"].shape

((4, 27, 59), (4, 3, 59, 41))

In [ ]:
if jnp.isnan(res["numeric_out"]).any():
    raise ValueError("NaN in numeric_out")
if jnp.isnan(res["categorical_out"]).any():
    raise ValueError("NaN in categorical_out")

In [24]:
ic.disable()

In [25]:
causal_mask = True
# time_series_regressor.train()

In [ ]:
def add_input_offsets(
    inputs: jnp.array, outputs: jnp.array, inputs_offset: int = 1
) -> jnp.array:
    inputs = inputs[:, :, inputs_offset:]
    tmp_null = jnp.full((inputs.shape[0], inputs.shape[1], inputs_offset), jnp.nan)
    inputs = jnp.concatenate([inputs, tmp_null], axis=2)
    nan_mask = jnp.isnan(inputs)
    inputs = jnp.where(nan_mask, jnp.zeros_like(inputs), inputs)
    # Add ext
    if outputs.ndim == inputs.ndim + 1:
        nan_mask_expanded = jnp.expand_dims(nan_mask, axis=-1)
        nan_mask_expanded = jnp.broadcast_to(nan_mask_expanded, outputs.shape)
    else:
        nan_mask_expanded = nan_mask

    # Apply mask to outputs
    outputs = jnp.where(nan_mask_expanded, jnp.zeros_like(outputs), outputs)

    return inputs, outputs, nan_mask
    # outputs = jnp.where(nan_mask, jnp.zeros_like(outputs), outputs)

    # return inputs, outputs, nan_mask


def numeric_loss(inputs, outputs, input_offset: int = 1):
    # print("Doing Numeric Loss")
    inputs, outputs, nan_mask = add_input_offsets(
        inputs=inputs, outputs=outputs, inputs_offset=input_offset
    )
    # print(f"Numeric: {inputs.shape=}, {outputs.shape=}, {nan_mask.shape=}")
    # TODO make loss SSL for values greater than 0.5 and MSE for values less than 0.5
    raw_loss = jnp.abs(outputs - inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss)
    # print(f"{masked_loss.shape=}, {nan_mask.shape=}")
    loss = masked_loss.sum() / (~nan_mask).sum()
    return loss


def categorical_loss(inputs, outputs, input_offset: int = 1):
    # print("Doing Categorical Loss")
    inputs, outputs, nan_mask = add_input_offsets(
        inputs=inputs, outputs=outputs, inputs_offset=input_offset
    )
    # print(f"Categorical: {inputs.shape=}, {outputs.shape=}, {nan_mask.shape=}")
    inputs = inputs.astype(jnp.int32)
    # print(f"Categorical Loss Shapes: {inputs.shape=}, {outputs.shape=}")
    raw_loss = optax.softmax_cross_entropy_with_integer_labels(outputs, inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss).mean()
    return masked_loss


@nnx.jit
def train_step(
    model: hp.TimeSeriesDecoder,
    inputs: dict,
    optimizer: nnx.Optimizer,
    metrics: nnx.MultiMetric,
):
    def loss_fn(model):
        res = model(
            numeric_inputs=inputs["numeric"],
            categorical_inputs=inputs["categorical"],
            deterministic=False,
        )

        numeric_loss_value = numeric_loss(inputs["numeric"], res["numeric_out"])
        categorical_loss_value = categorical_loss(
            inputs["categorical"], res["categorical_out"]
        )
        loss = numeric_loss_value + categorical_loss_value
        return loss, (numeric_loss_value, categorical_loss_value)

    grad_fn = nnx.value_and_grad(loss_fn, has_aux=True)
    # grad_fn = nnx.value_and_grad(loss_fn, has_aux=False)
    # loss, grads = grad_fn(model)
    (loss, (numeric_loss_value, categorical_loss_value)), grads = grad_fn(model)
    metrics.update(
        loss=loss,
        numeric_loss=numeric_loss_value,
        categorical_loss=categorical_loss_value,
    )

    optimizer.update(grads)


metric_history = {
    "loss": [],
    "numeric_loss": [],
    "categorical_loss": [],
}

learning_rate = 1e-3
momentum = 0.9
optimizer = nnx.Optimizer(time_series_regressor, optax.adamw(learning_rate, momentum))
metrics = nnx.MultiMetric(
    loss=nnx.metrics.Average("loss"),
    categorical_loss=nnx.metrics.Average("categorical_loss"),
    numeric_loss=nnx.metrics.Average("numeric_loss"),
)
writer_name = "DoesThisWork"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = writer_time + writer_name
summary_writer = SummaryWriter("runs/" + model_name)


metrics_history = {"train_loss": []}
train_data_loader = DataLoader(train_ds, batch_size=16, shuffle=True)

for step, batch in enumerate(tqdm(train_data_loader)):
    batch = {"numeric": jnp.array(batch[0]), "categorical": jnp.array(batch[1])}
    train_step(time_series_regressor, batch, optimizer, metrics)
    for metric, value in metrics.compute().items():
        # Only shows `loss`

        metric_history[metric].append(value)
        if jnp.isnan(value).any():
            print("Nan Values")
        else:
            print("No Nan Values")
            # raise ValueError("Nan Values")
        summary_writer.add_scalar(f"train/{metric}", np.array(value), step)
    metrics.reset()
    # print(f"Step {step}")

In [ ]:
# import orbax.checkpoint as ocp

# ckpt_dir = ocp.test_utils.erase_and_create_empty("/tmp/my-checkpoints1/")
# _, state = nnx.split(time_series_regressor)
# state = state.to_pure_dict()
# # nnx.display(state)

# checkpointer = ocp.StandardCheckpointer()
# checkpointer.save(ckpt_dir / "state", state)

In [ ]:
# import jax.numpy as jnp
# import jax.tree_util as jtu


# # Print the structure and dtypes
# def print_dtype_tree(tree):
#     def get_dtype(x):
#         if hasattr(x, "dtype"):
#             return x.dtype
#         return type(x)

#     return jtu.tree_map(get_dtype, tree)


# print("State structure and dtypes:")
# print(jtu.tree_map(lambda x: type(x), state))
# print("\nDetailed dtype information:")
# print(print_dtype_tree(state))

In [27]:
@dataclass
class Results:
    numeric_out: jnp.array
    categorical_out: jnp.array
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array


def return_results(model, dataset, idx=0, mask_start: int = None):
    numeric_inputs, categorical_inputs = dataset[idx]
    if mask_start:
        numeric_inputs = numeric_inputs[:, :mask_start]
        categorical_inputs = categorical_inputs[:, :mask_start]
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])
    out = time_series_regressor(
        numeric_inputs=numeric_inputs, categorical_inputs=categorical_inputs
    )
    numeric_out, categorical_out = out["numeric_out"], out["categorical_out"]
    return Results(numeric_out, categorical_out, numeric_inputs, categorical_inputs)


x = return_results(time_series_regressor, train_ds, 0)
x.categorical_out.shape

(1, 3, 59, 41)

In [28]:
causal_mask = False
causal_mask = True


def process_results(arr: jnp.array, col_names: list, config: hp.TimeSeriesConfig):
    arr = jnp.squeeze(arr)
    if arr.ndim == 3:
        # Check if there is a logit array for example if there are 3 dims then the
        # last dim is the logit array. We need to get the argmax of the last dim
        # to get the actual values of the array and replace the logit array with the
        # actual values
        arr = jnp.argmax(arr, axis=-1)
    df = pd.DataFrame(arr.T)
    df.columns = col_names
    return df


@dataclass
class DFComparison:
    input_df: pd.DataFrame
    output_df: pd.DataFrame


def show_results_df(
    model, time_series_config, dataset, idx: int = 0, mask_start: int = None
):
    results = return_results(model, dataset, idx=idx, mask_start=mask_start)

    input_categorical = process_results(
        results.categorical_inputs,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    input_numeric = process_results(
        results.numeric_inputs,
        time_series_config.numeric_col_tokens,
        time_series_config,
    )
    output_categorical = process_results(
        results.categorical_out,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    output_numeric = process_results(
        results.numeric_out, time_series_config.numeric_col_tokens, time_series_config
    )
    input_df = pd.concat([input_categorical, input_numeric], axis=1)
    output_df = pd.concat([output_categorical, output_numeric], axis=1)

    return DFComparison(input_df, output_df)


df_comp = show_results_df(
    model=time_series_regressor,
    time_series_config=time_series_config,
    dataset=train_ds,
    idx=0,
)

In [29]:
df_comp.output_df.loc[:, time_series_config.categorical_col_tokens].tail()

,n_planets,acceleration_x,acceleration_y
54,37,33,33
55,37,33,33
56,37,33,33
57,37,33,33
58,37,33,33


In [31]:
df_comp.output_df.loc[:, time_series_config.numeric_col_tokens].tail()

,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,planet0_e,...,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass
54,8.799849,-0.063316,0.099225,-0.145077,0.176468,-0.271472,0.016535,2.661092,1.309041,0.936999,...,0.198029,2.713041,1.349090,0.874316,-0.166074,0.179713,2.538124,1.511868,1.049518,9.145997
55,8.792229,-0.057472,0.102335,-0.141220,0.176780,-0.280962,0.014335,2.666697,1.315591,0.941196,...,0.201190,2.718580,1.355090,0.876884,-0.169938,0.182239,2.544012,1.514011,1.055833,9.161053
56,8.522557,-0.060992,0.100365,-0.135474,0.171987,-0.269103,0.012860,2.584574,1.275140,0.911134,...,0.198084,2.633257,1.313677,0.849998,-0.163366,0.173668,2.464268,1.467881,1.021853,8.871813
57,8.809786,-0.060079,0.093415,-0.156883,0.168826,-0.289809,0.016125,2.657580,1.314180,0.941469,...,0.185946,2.713940,1.353394,0.873902,-0.183492,0.184239,2.539721,1.513540,1.053416,9.155434
58,8.713181,-0.046250,0.084833,-0.155417,0.166590,-0.267556,0.036527,2.633353,1.298787,0.931455,...,0.169708,2.692102,1.335361,0.865630,-0.180343,0.189877,2.514788,1.503370,1.039747,9.070271


In [32]:
def plot_planets(df_pred: pd.DataFrame, df_actual: pd.DataFrame, column: str, offset=0):
    plt.figure(figsize=(15, 10))
    plt.plot(df_pred[column], label="Autogregressive")
    plt.plot(df_actual[column], label="Actual")
    plt.title(f"{column} Predictions")
    plt.legend()
    # Show ticks and grid lines every 1 step
    plt.xticks(np.arange(0, len(df_pred), 1))
    plt.grid()
    # add black line at 0 on the y axis to show the difference
    plt.axhline(0, color="black")
    plt.show()

In [33]:
jnp.array([True, True, False, False, True]).shape

(5,)

In [34]:
x = jnp.ones((5, 20))
print(x.shape)
xx = x.at[jnp.array([True, True, False, False, True]), :].set(0)
xx

(5, 20)


Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [41]:
@dataclass
class AutoRegressiveResults:
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array

    @classmethod
    def from_ds(cls, ds: hp.TimeSeriesDS, idx: int, stop_idx: int = 10):
        inputs = ds[idx]
        numeric_inputs = inputs[0][:, :stop_idx]
        categorical_inputs = inputs[1][:, :stop_idx]
        return cls(numeric_inputs, categorical_inputs)


def auto_regressive_predictions(
    model: hp.TimeSeriesDecoder,
    inputs: AutoRegressiveResults,
) -> jnp.array:
    numeric_inputs = inputs.numeric_inputs
    categorical_inputs = inputs.categorical_inputs

    # get the first row that contains all nan vales
    # if nan_rows_start >= stop_idx:
    #     return inputs
    # numeric_inputs = inputs.numeric_inputs
    # categorical_inputs = inputs.categorical_inputs
    numeric_nan_columns = jnp.isnan(numeric_inputs).all(axis=1)

    categorical_nan_columns = jnp.isnan(categorical_inputs).all(axis=1)

    print(
        f"numeric_inputs shape: {numeric_inputs.shape}, categorical_inputs shape: {categorical_inputs.shape}"
    )

    # Expand first dim to make it a batch
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])
    print(
        f"numeric_inputs shape: {numeric_inputs.shape}, categorical_inputs shape: {categorical_inputs.shape}"
    )

    outputs = time_series_regressor(
        numeric_inputs=numeric_inputs, categorical_inputs=categorical_inputs
    )
    numeric_out = outputs["numeric_out"]  #  jnp.squeeze(outputs["numeric_out"])
    categorical_out = outputs[
        "categorical_out"
    ]  # jnp.squeeze(outputs["categorical_out"])
    print(
        f"numeric type: {numeric_out.dtype}, categorical type: {categorical_out.dtype}"
    )
    categorical_out = jnp.argmax(categorical_out, axis=-1)

    final_numeric_row = np.array(numeric_out[:, -1])
    final_numeric_row = final_numeric_row[:, None]  # New axis

    final_categorical_row = np.array(categorical_out[:, -1])
    final_categorical_row = final_categorical_row[:, None]  # New axis
    print(f"{numeric_inputs.shape=}, {final_numeric_row.shape=}")
    numeric_inputs = jnp.concatenate([numeric_inputs, final_numeric_row], axis=1)
    categorical_inputs = jnp.concatenate(
        [categorical_inputs, final_categorical_row], axis=1
    )
    numeric_inputs = numeric_inputs.at[jnp.array(numeric_nan_columns)].set(jnp.nan)
    categorical_inputs = categorical_inputs.at[jnp.array(categorical_nan_columns)].set(
        jnp.nan
    )
    inputs = (numeric_inputs, categorical_inputs)

    return inputs
    # return auto_regressive_predictions(state, inputs, stop_idx)


test_inputs = AutoRegressiveResults.from_ds(train_ds, 0, 10)
# inputs_test = train_ds[0]
# test_numeric = inputs_test[0]
# test_categorical = inputs_test[1]
# print(inputs_test.shape)
for i in trange(21):
    print(f"Step {i}")
    inputs_test = auto_regressive_predictions(time_series_regressor, test_inputs)

# x = auto_regressive_predictions(state, test_ds[0], 10)

  0%|          | 0/21 [00:00<?, ?it/s]

Step 0
numeric_inputs shape: (27, 10), categorical_inputs shape: (3, 10)
numeric_inputs shape: (1, 27, 10), categorical_inputs shape: (1, 3, 10)
numeric type: float32, categorical type: float32
numeric_inputs.shape=(1, 27, 10), final_numeric_row.shape=(1, 1, 10)


IndexError: boolean index did not match shape of indexed array in index 0: got (27,), expected (1,)

In [ ]:
test_inputs = AutoRegressiveResults.from_ds(train_ds, 0, 10)

test_inputs.numeric_inputs.shape

In [ ]:
[train_ds[0][0][:, :10].shape, train_ds[0][1][:, :10].shape]

In [ ]:
time_series_regressor(
    numeric_inputs=jnp.array([train_ds[0][0]]),
    # numeric_inputs=jnp.array([train_ds[0][0][:, :10]]),
    categorical_inputs=jnp.array([train_ds[0][1]]),
    # categorical_inputs=jnp.array([train_ds[0][1][:, :10]]),
)

In [ ]:
res1 = time_series_regressor(
    numeric_inputs=jnp.array([train_ds[0][0][:, :10]]),
    categorical_inputs=jnp.array([train_ds[0][1][:, :10]]),
)
res1

In [ ]:
jnp.isnan(res1["numeric_out"]).any()

In [ ]:
test_inputs.numeric_inputs.shape